In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests

In [3]:
import datetime

In [4]:
import base64

In [5]:
client_id = 'a626486e1d8b4b7294b0d2eff0046d99'
client_secret = '9b39b256a71246a3836d8ad9f871c330'

In [6]:
# do a lookup for a token
# this token is for future requests

In [7]:
client_creds = f"{client_id}:{client_secret}"
type(client_creds)

str

In [8]:
client_creds_b64 = base64.b64encode(client_creds.encode())
type(client_creds_b64)

bytes

In [9]:
# base64.b64deocde(client_creds_b64)

In [10]:
token_url = "https://accounts.spotify.com/api/token"
method = "POST"
token_data = {
    "grant_type": "client_credentials"
    
}
token_headers = {
    "Authorization": f"Basic {client_creds_b64.decode()}" # <base64 encoded client_id:client_secret>
}


In [11]:
r = requests.post(token_url, data=token_data, headers=token_headers)
print(r.json())
valid_request = r.status_code in range(200, 299)

{'access_token': 'BQAdEXsZUeXe0IL3_8kiEBVYbu0RSAFhWbI8WDRy-wvRmPIFzUGxsAmnhKSA1XQlrdsCHsnlSET-53Iqvv4', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}


In [12]:
if valid_request:
    token_response_data = r.json()
    now = datetime.datetime.now()
    access_token = token_response_data['access_token']
    expires_in = token_response_data['expires_in'] # seconds
    expires = now + datetime.timedelta(seconds=expires_in)
    did_expire = expires < now
 

In [13]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 

In [15]:
client_id = 'a626486e1d8b4b7294b0d2eff0046d99'
client_secret = '9b39b256a71246a3836d8ad9f871c330'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [16]:
#Extract data from Spotify playlist
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

ids = getTrackIDs('spotify', '7CpxSiBfVue7mJxLO8BeO0')

In [17]:
print(len(ids))
print(ids)

40
['3FskQrDXcY24ur2fCvz35O', '4gMXTO4f8hCe6jVhujwpUd', '55r3LnrYYQBZ2pkls4ZTnB', '07XZZTucIfMyOSnkF0GPWJ', '2OluGQ4FdEhpOjJGveO3b2', '0b9oOr2ZgvyQu88wzixux9', '0ECs7wpW9157Tk5yBUGbE0', '44gRhRi2OhEf7moAUj6MD1', '4R9CKeS4mIiOz417aLtHUv', '4QV4EEYszD7HGrTo6PzX9K', '5oZps6mYqU5s7A4WjUZggJ', '3rDZ6pStaJhzk1mXjDyOfR', '1b4lViVfvHfLnNnEoQQSRI', '5We8lGFNpA1IkHAfIJU9XM', '2riMQleCMgZoZRLbhBHuY6', '3JBvJ9PwrYA1oeUzI3FMRI', '6k0DJqW2v2e4809Z1zN0YU', '1tXCF7TwDnuHRgwQ63FeM5', '5YqYLur57vzjL0Ts8fk0G2', '6SKP6d0DFOBemCvThbuqUj', '22DHmfJa31hKpc2Lg0gVBV', '5nIUYBNePm0jgOh4nsnaRK', '55r3LnrYYQBZ2pkls4ZTnB', '78tw2cG41XXZSUD1RisQ5q', '3SMd9iGyHF47xNZw3SXFPE', '6B3H7aue15Ntp6sfQaH7wE', '2kQ8Ge6E9ycl0bcYaAiMYg', '25qbZLtqkvcbIjK7ekXong', '2qzUpSVI4NnPyWxbXwumTj', '4CPszpSnlbgnQLIQYyzIR2', '7aBxcRw77817BrkdPChAGY', '0ojNk5JVukyByGIps1Sw42', '5hkO00MFefaYioY4Ridxpf', '56S4e7Ix1GnDEoAxwoaLYp', '2jLDRXGpa9yJ9fuhySSV8P', '3pk8pThFHSD1fjISYCGWeI', '1v49ImdfM7a8TcnBzwHhLY', '5HvVoij9zaVGoyspI2LlwU', '2BEuUkh

In [18]:
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  valence = features[0]['valence']

  track = [name, album, artist, release_date, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, valence]
  return track

In [19]:
# loop over track ids 
tracks = []
for i in range(len(ids)):
  time.sleep(.5)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

# create dataset
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])
df.to_csv("spotify.csv", sep = ',')

In [20]:
import pandas as pd

In [21]:
# loop over track ids 
tracks = []
for i in range(len(ids)):
  time.sleep(.5)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

# create dataset
df = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])
df.to_csv("spotify.csv", sep = ',')

In [22]:
print (df)

                                                 name  \
0                                                  Ye   
1                                                 Ole   
2                       Sorrow Tears and Blood - Edit   
3                                             Dangote   
4                      I NEED YOU TO (BREONNA TAYLOR)   
5                                     This Is America   
6                                             Alright   
7                                  The Bigger Picture   
8                                     They Don't Care   
9   Don't Shoot (feat. Rick Ross, Fabolous, Diddy,...   
10                                           Lockdown   
11                                              I Cry   
12                             JUNE 5th / QueenZnGodZ   
13                            Black (feat. A$AP Ferg)   
14  Mr. Officer (feat. Queen Naija and members of ...   
15                               Otherside Of America   
16                             

In [23]:
df

,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence
0,Ye,Outside,Burna Boy,2018-01-26,231618,72,0.503,0.01810,0.503,0.592,0.000140,0.1690,-5.923,0.4440,201.850,4,0.3440
1,Ole,Celia,Tiwa Savage,2020-08-27,183653,44,0.800,0.59200,0.800,0.801,0.000055,0.5600,-6.315,0.2140,99.936,4,0.6840
2,Sorrow Tears and Blood - Edit,Sorrow Tears and Blood,Fela Kuti,2020-09-03,463052,26,0.851,0.14600,0.851,0.854,0.000458,0.2230,-5.140,0.1990,101.376,4,0.8310
3,Dangote,African Giant,Burna Boy,2019-07-25,225404,52,0.539,0.50500,0.539,0.651,0.000002,0.0802,-5.017,0.4030,67.169,5,0.8000
4,I NEED YOU TO (BREONNA TAYLOR),THE PANDEMIC PROJECT,Tobe Nwigwe,2020-08-02,41659,47,0.881,0.00796,0.881,0.298,0.008430,0.1120,-8.626,0.0738,120.034,4,0.3640
5,This Is America,This Is America,Childish Gambino,2018-05-06,225773,74,0.854,0.11700,0.854,0.463,0.000000,0.3540,-6.159,0.1370,120.024,4,0.5490
6,Alright,Alright,Kendrick Lamar,2015-10-16,194490,54,0.783,0.08640,0.783,0.761,0.000000,0.6430,-5.530,0.1950,110.033,4,0.6740
7,The Bigger Picture,The Bigger Picture,Lil Baby,2020-06-12,252728,77,0.590,0.01270,0.590,0.618,0.000000,0.2440,-5.756,0.3340,96.459,3,0.1530
8,They Don't Care,They Don't Care,Lloyd,2020-09-22,183427,37,0.591,0.06840,0.591,0.640,0.000021,0.4080,-7.165,0.1170,112.216,4,0.2590
9,"Don't Shoot (feat. Rick Ross, Fabolous, Diddy,...","Don't Shoot (feat. Rick Ross, 2 Chainz, Diddy,...",The Game,2014-08-27,367151,49,0.699,0.07560,0.699,0.491,0.000000,0.6100,-9.911,0.2180,81.095,4,0.2400


In [24]:
#Cleaning data of BLM playlist
blm = pd.read_csv('spotify.csv')

In [25]:
blm.columns

Index(['Unnamed: 0', 'name', 'album', 'artist', 'release_date', 'length',
       'popularity', 'danceability', 'acousticness', 'danceability.1',
       'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
       'tempo', 'time_signature', 'valence'],
      dtype='object')

In [26]:
#Drop unwanted columns
to_drop = ['Unnamed: 0',
           'length',
           'album',
           'release_date',
           'danceability.1',
           'valence',
           'time_signature']
blm.drop(columns=to_drop, inplace=True) 

In [27]:
blm

,name,artist,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo
0,Ye,Burna Boy,72,0.503,0.01810,0.592,0.000140,0.1690,-5.923,0.4440,201.850
1,Ole,Tiwa Savage,44,0.800,0.59200,0.801,0.000055,0.5600,-6.315,0.2140,99.936
2,Sorrow Tears and Blood - Edit,Fela Kuti,26,0.851,0.14600,0.854,0.000458,0.2230,-5.140,0.1990,101.376
3,Dangote,Burna Boy,52,0.539,0.50500,0.651,0.000002,0.0802,-5.017,0.4030,67.169
4,I NEED YOU TO (BREONNA TAYLOR),Tobe Nwigwe,47,0.881,0.00796,0.298,0.008430,0.1120,-8.626,0.0738,120.034
5,This Is America,Childish Gambino,74,0.854,0.11700,0.463,0.000000,0.3540,-6.159,0.1370,120.024
6,Alright,Kendrick Lamar,54,0.783,0.08640,0.761,0.000000,0.6430,-5.530,0.1950,110.033
7,The Bigger Picture,Lil Baby,77,0.590,0.01270,0.618,0.000000,0.2440,-5.756,0.3340,96.459
8,They Don't Care,Lloyd,37,0.591,0.06840,0.640,0.000021,0.4080,-7.165,0.1170,112.216
9,"Don't Shoot (feat. Rick Ross, Fabolous, Diddy,...",The Game,49,0.699,0.07560,0.491,0.000000,0.6100,-9.911,0.2180,81.095


In [28]:
#Extract data from Spotify playlist
def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
    return ids

ids = getTrackIDs('Spotify', '37i9dQZF1DX6VCVxWsoEjE')

In [29]:
print(len(ids))
print(ids)

56
['4WMec4QUeLb7O9yqZleFxg', '5KJ5TkNBbLDrK88oP2enS8', '1vDKPP3DcTfgiDp3GUZBk5', '5eDRQNfBueQB1EWBodrLjt', '3teEkXkxQI91s2kRvRaZN8', '3vsVY7KHvnA8jkKiuj3Uxi', '486DiCBGlYNjazRAOUeffI', '0uaM2q4FQOswkHruAf37vg', '48XrCuFxSkXfaF1PcZ8ODb', '2Z0W5j0qWAam69m68wPdIl', '0oXAx3Gj3yW6C8DgoSrRis', '7Eb5AYiiGWItBNddlmxvnh', '74JGKtGEQjx2gXZgSR4v93', '0JjkKdoYG9SSK2032LfcY4', '4VfaYaXYPhUJZoioVTzL06', '15KN4rvxCnsge5cCBOzRxe', '7jvGwqJkhJ5eNAlRsKRNJI', '6FA2rRogpLIOZ0ffho4uPL', '1svJzK3MXaqwGf5PyPmx8w', '0xyfWNeHT9Tg3MowNc9ISc', '3XlzV8aKsldkPtm7Wiurcd', '78E8UFcMXaqHcVszojrRN3', '61zjDVLIGjiXwMas5CsoOs', '6N1rVxZ2pYKDcDOj5x5XhQ', '2pMqPNZkNPXgLKZjlnwLtd', '4Dv9tGgzP9Pxq08e6hSmMo', '3shWwjQSPLvLbRhxLbEbpg', '2JByEiUMZYsgE48DpXp1WD', '1I3ElbZ856vokHhyvsyvng', '3XVghykPqPOQczam54CCaE', '0pFGnSemrTFHFN4PPbGMvS', '0qrOLBK3bCzZs9jYl9zMtu', '52xzRsYHAGrKYuWte06LuM', '16IT9SCs4vy4qmCfuujX7M', '4aFbsLHNFfgUNLeEbTqVxM', '2nsQCxj1f284EfY1br4SuH', '0EV7zZsYIVWSdpPnHnEkR6', '4FpRhJ7lOtetHL1zwZSYuP', '70YwAas

In [30]:
def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  release_date = meta['album']['release_date']
  length = meta['duration_ms']
  popularity = meta['popularity']

  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  valence = features[0]['valence']

  track = [name, album, artist, release_date, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, valence]
  return track

In [31]:
# loop over track ids 
tracks = []
for i in range(len(ids)):
  time.sleep(.5)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

# create dataset
df1 = pd.DataFrame(tracks, columns = ['name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])

In [32]:
print (df1)

                                                 name  \
0                                             El Beso   
1                                               Seivi   
2                                  Cómo Te Lo Explico   
3                                           Pe U Te O   
4                                                Tuya   
5                                        Veni a Sudar   
6                                         Ponte Perra   
7                                           La Dealer   
8                                                Isla   
9                                         LIBRE DE MI   
10                                              Tebas   
11                                 A quién le importa   
12                                             Espada   
13                                              Fuego   
14                                                 IR   
15                               La Revolución Sexual   
16                             

In [33]:
df1

,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence
0,El Beso,Las Yeguas Del Apocalipsis,Sudor Marika,2017-11-08,73665,20,0.658,0.060400,0.658,0.166,0.000000,0.3590,-15.821,0.9480,92.559,4,0.578
1,Seivi,Seivi,Kumbia Queers,2020-06-28,239710,25,0.709,0.051300,0.709,0.898,0.000003,0.1010,-4.669,0.0450,95.016,4,0.794
2,Cómo Te Lo Explico,Cómo Te Lo Explico,Chocolate Remix,2020-09-25,201504,22,0.784,0.066900,0.784,0.865,0.000004,0.0991,-6.028,0.0817,95.969,4,0.742
3,Pe U Te O,Re Fuerte,La Queen,2019-11-29,194279,16,0.881,0.183000,0.881,0.545,0.000182,0.1030,-8.729,0.1640,118.001,4,0.573
4,Tuya,Bailemos Juntas,Ibiza Pareo,2018-05-11,270123,27,0.618,0.019400,0.618,0.680,0.009100,0.0725,-9.631,0.0379,120.009,4,0.473
5,Veni a Sudar,Las Yeguas Del Apocalipsis,Sudor Marika,2017-11-08,185552,28,0.559,0.434000,0.559,0.662,0.000000,0.0907,-6.215,0.0506,93.192,4,0.689
6,Ponte Perra,111,Pabllo Vittar,2020-03-24,133373,43,0.733,0.069100,0.733,0.662,0.000021,0.0902,-6.705,0.2420,175.853,4,0.382
7,La Dealer,ReBeba MiXXXtape,SASHA SATHYA,2019-03-07,152132,20,0.782,0.025700,0.782,0.479,0.090300,0.0991,-8.071,0.0827,169.915,4,0.239
8,Isla,Mar,Leo García,2001-01-01,175560,34,0.751,0.106000,0.751,0.782,0.000021,0.4170,-5.709,0.0294,140.001,4,0.967
9,LIBRE DE MI,TODA,BIFE,2016-04-23,222140,32,0.506,0.760000,0.506,0.511,0.000031,0.1220,-10.260,0.0342,88.915,4,0.576


In [34]:
df1.to_csv("spotify1.csv", sep = ',')
#Cleaning data of LGBTQ playlist
LGBTQ = pd.read_csv('spotify1.csv')

In [35]:
LGBTQ.columns

Index(['Unnamed: 0', 'name', 'album', 'artist', 'release_date', 'length',
       'popularity', 'danceability', 'acousticness', 'danceability.1',
       'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness',
       'tempo', 'time_signature', 'valence'],
      dtype='object')

In [36]:
LGBTQ

,Unnamed: 0,name,album,artist,release_date,length,popularity,danceability,acousticness,danceability.1,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence
0,0,El Beso,Las Yeguas Del Apocalipsis,Sudor Marika,2017-11-08,73665,20,0.658,0.060400,0.658,0.166,0.000000,0.3590,-15.821,0.9480,92.559,4,0.578
1,1,Seivi,Seivi,Kumbia Queers,2020-06-28,239710,25,0.709,0.051300,0.709,0.898,0.000003,0.1010,-4.669,0.0450,95.016,4,0.794
2,2,Cómo Te Lo Explico,Cómo Te Lo Explico,Chocolate Remix,2020-09-25,201504,22,0.784,0.066900,0.784,0.865,0.000004,0.0991,-6.028,0.0817,95.969,4,0.742
3,3,Pe U Te O,Re Fuerte,La Queen,2019-11-29,194279,16,0.881,0.183000,0.881,0.545,0.000182,0.1030,-8.729,0.1640,118.001,4,0.573
4,4,Tuya,Bailemos Juntas,Ibiza Pareo,2018-05-11,270123,27,0.618,0.019400,0.618,0.680,0.009100,0.0725,-9.631,0.0379,120.009,4,0.473
5,5,Veni a Sudar,Las Yeguas Del Apocalipsis,Sudor Marika,2017-11-08,185552,28,0.559,0.434000,0.559,0.662,0.000000,0.0907,-6.215,0.0506,93.192,4,0.689
6,6,Ponte Perra,111,Pabllo Vittar,2020-03-24,133373,43,0.733,0.069100,0.733,0.662,0.000021,0.0902,-6.705,0.2420,175.853,4,0.382
7,7,La Dealer,ReBeba MiXXXtape,SASHA SATHYA,2019-03-07,152132,20,0.782,0.025700,0.782,0.479,0.090300,0.0991,-8.071,0.0827,169.915,4,0.239
8,8,Isla,Mar,Leo García,2001-01-01,175560,34,0.751,0.106000,0.751,0.782,0.000021,0.4170,-5.709,0.0294,140.001,4,0.967
9,9,LIBRE DE MI,TODA,BIFE,2016-04-23,222140,32,0.506,0.760000,0.506,0.511,0.000031,0.1220,-10.260,0.0342,88.915,4,0.576


In [37]:
#Drop unwanted columns
to_drop = ['Unnamed: 0',
           'length',
           'album',
           'release_date',
           'danceability.1',
           'valence',
           'time_signature']
LGBTQ.drop(columns=to_drop, inplace=True) 

In [38]:
LGBTQ.head(40)

,name,artist,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo
0,El Beso,Sudor Marika,20,0.658,0.060400,0.166,0.000000,0.3590,-15.821,0.9480,92.559
1,Seivi,Kumbia Queers,25,0.709,0.051300,0.898,0.000003,0.1010,-4.669,0.0450,95.016
2,Cómo Te Lo Explico,Chocolate Remix,22,0.784,0.066900,0.865,0.000004,0.0991,-6.028,0.0817,95.969
3,Pe U Te O,La Queen,16,0.881,0.183000,0.545,0.000182,0.1030,-8.729,0.1640,118.001
4,Tuya,Ibiza Pareo,27,0.618,0.019400,0.680,0.009100,0.0725,-9.631,0.0379,120.009
5,Veni a Sudar,Sudor Marika,28,0.559,0.434000,0.662,0.000000,0.0907,-6.215,0.0506,93.192
6,Ponte Perra,Pabllo Vittar,43,0.733,0.069100,0.662,0.000021,0.0902,-6.705,0.2420,175.853
7,La Dealer,SASHA SATHYA,20,0.782,0.025700,0.479,0.090300,0.0991,-8.071,0.0827,169.915
8,Isla,Leo García,34,0.751,0.106000,0.782,0.000021,0.4170,-5.709,0.0294,140.001
9,LIBRE DE MI,BIFE,32,0.506,0.760000,0.511,0.000031,0.1220,-10.260,0.0342,88.915


In [39]:
import os

In [40]:
df1.to_csv(r'D:\download\spotify1.csv', index = False)

In [41]:
df.to_csv(r'D:\download\spotify0.csv', index = False)